<a href="https://colab.research.google.com/github/JongHyun9999/AutoEncoder_augmentation/blob/LSTM_CV/lstmAE_inter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import random
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import datetime
# your_csv_path = './open'
your_csv_path = './drive/MyDrive'

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [15]:
CFG = {
    'TRAIN_WINDOW_SIZE':90, # 90일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':5,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':3178,
    'SEED':41
}

In [16]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

데이터 불러오기

In [17]:
train_data = pd.read_csv(your_csv_path + '/train.csv').drop(columns=['ID', '제품'])
# sales_data = pd.read_csv(your_csv_path + '/sales.csv').drop(columns=['ID', '제품'])
# keyword_data = pd.read_csv(your_csv_path + './brand_keyword_cnt.csv')
# product_info_data = pd.read_csv(your_csv_path + './product_info.csv')

In [18]:
train_data.head()

,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,1,3,2,0,0,2,0
2,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# sales_data.head()

데이터 전처리

결측치 처리

In [20]:
print("train DataFrame 크기 : ", train_data.shape);
train_data.info()
# print("sales DataFrame 크기 : ", sales_data.shape);
# sales_data.info()

train DataFrame 크기 :  (15890, 463)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15890 entries, 0 to 15889
Columns: 463 entries, 대분류 to 2023-04-04
dtypes: int64(459), object(4)
memory usage: 56.1+ MB


In [21]:
def findOutlier(product_series):
    # 제품별 판매기록이 담긴 Series 데이터를 받아, 상대변화율을 계산하여 급증데이터를 완만하게합니다.

    # 상대변화율 계산
    product_series = product_series + 1
    relative_change_rate = (product_series - product_series.shift(1)) / product_series.shift(1)

    # 상대변화율이 50% 이상인 데이터 찾기.
    outliers = relative_change_rate[relative_change_rate >= 0.3].index
    if(outliers.__contains__('2023-04-04')):
        outliers = outliers.drop('2023-04-04')

    # 급증데이터를 완만하게 처리
    for outlier in outliers:
        date = datetime.datetime.strptime(outlier, '%Y-%m-%d')
        # 1일 후의 날짜
        next_day = (date + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
        # 1일 전의 날짜
        prev_day = (date - datetime.timedelta(days=1)).strftime('%Y-%m-%d')

        product_series.loc[outlier] = ( product_series.loc[next_day] + product_series.loc[prev_day] ) /2


    return product_series - 1

In [22]:
for i in tqdm(range(train_data.shape[0])):
    train_data.iloc[i,4:] = findOutlier(train_data.iloc[i,4:])

  0%|          | 0/15890 [00:00<?, ?it/s]

In [23]:
# # 데이터 스케일링
# #
# scale_max_dict = {}
# scale_min_dict = {}

# for idx in tqdm(range(len(train_data))):
#     maxi = np.max(train_data.iloc[idx,4:])
#     mini = np.min(train_data.iloc[idx,4:])

#     if maxi == mini :
#         train_data.iloc[idx,4:] = 0
#     else:
#         train_data.iloc[idx,4:] = (train_data.iloc[idx,4:] - mini) / (maxi - mini)

#     scale_max_dict[idx] = maxi
#     scale_min_dict[idx] = mini

# 숫자형 변수들의 min-max scaling을 수행하는 코드입니다.
numeric_cols = train_data.columns[4:]
# 칵 column의 min 및 max 계산
min_values = train_data[numeric_cols].min(axis=1)
max_values = train_data[numeric_cols].max(axis=1)
# 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 1로 대체
ranges = max_values - min_values
ranges[ranges == 0] = 1
# min-max scaling 수행
train_data[numeric_cols] = (train_data[numeric_cols].subtract(min_values, axis=0)).div(ranges, axis=0)
# max와 min 값을 dictionary 형태로 저장
scale_min_dict = min_values.to_dict()
scale_max_dict = max_values.to_dict()

In [24]:
# Label Encoding
# 제품의 분류코드인 String을 숫자로 인코딩
label_encoder = LabelEncoder()
categorical_columns = ['대분류', '중분류', '소분류', '브랜드']

for col in categorical_columns:
    label_encoder.fit(train_data[col])
    train_data[col] = label_encoder.transform(train_data[col])

In [25]:
def make_train_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
    data : 일별 판매량
    train_size : 학습에 활용할 기간
    predict_size : 추론할 기간
    '''
    num_rows = len(data)
    window_size = train_size + predict_size

    input_data = np.empty((num_rows * (len(data.columns) - window_size + 1), train_size, len(data.iloc[0, :4]) + 1))
    target_data = np.empty((num_rows * (len(data.columns) - window_size + 1), predict_size))

    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, 4:])

        for j in range(len(sales_data) - window_size + 1):
            window = sales_data[j : j + window_size]
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
            input_data[i * (len(data.columns) - window_size + 1) + j] = temp_data
            target_data[i * (len(data.columns) - window_size + 1) + j] = window[train_size:]

    return input_data, target_data

In [26]:
def make_predict_data(data, train_size=CFG['TRAIN_WINDOW_SIZE']):
    '''
    평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
    data : 일별 판매량
    train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간)
    '''
    num_rows = len(data)

    input_data = np.empty((num_rows, train_size, len(data.iloc[0, :4]) + 1))

    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, -train_size:])

        window = sales_data[-train_size : ]
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
        input_data[i] = temp_data

    return input_data

In [27]:
train_input, train_target = make_train_data(train_data)
test_input = make_predict_data(train_data)

  0%|          | 0/15890 [00:00<?, ?it/s]

  0%|          | 0/15890 [00:00<?, ?it/s]

In [28]:
# Train / Validation Split
data_len = len(train_input)
val_input = train_input[-int(data_len*0.2):]
val_target = train_target[-int(data_len*0.2):]
train_input = train_input[:-int(data_len*0.2)]
train_target = train_target[:-int(data_len*0.2)]

In [29]:
train_input.shape, train_target.shape, val_input.shape, val_target.shape, test_input.shape

((4487336, 90, 5),
 (4487336, 21),
 (1121834, 90, 5),
 (1121834, 21),
 (15890, 90, 5))

Custom Dataset

In [30]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])

    def __len__(self):
        return len(self.X)

In [31]:
train_dataset = CustomDataset(train_input, train_target)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0, drop_last=True)

val_dataset = CustomDataset(val_input, val_target)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0, drop_last=True)

모델 선언

In [32]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, sequence_length, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.hidden_size = hidden_size
        self.sequence_length = sequence_length
        self.encoder1 = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.encoder2 = nn.LSTM(hidden_size, hidden_size//2, batch_first=True)

    def init_hidden(self, batch_size, device):
        # Initialize hidden state and cell state
        return (torch.zeros(1, batch_size, self.hidden_size, device=device),
                torch.zeros(1, batch_size, self.hidden_size, device=device))

    def forward(self, x):
        # x shape: (B, TRAIN_WINDOW_SIZE, 5)
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size, x.device)

        # x = (Batch_size, TRAIN_WINDOW_SIZE, 5)
        encoded_output1, _ = self.encoder1(x, hidden)
        _, hidden_state = self.encoder2(encoded_output1)
        reshaped_tensor = hidden_state[0].reshape(self.batch_size, 1, self.hidden_size//2)
        expanded_tensor = reshaped_tensor.expand(-1, self.sequence_length, -1)
        return expanded_tensor


class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.decoder1 = nn.LSTM(hidden_size//2, hidden_size//2, batch_first=True)
        self.decoder2 = nn.LSTM(hidden_size//2, hidden_size, batch_first=True)
        # self.timedistributed = TimeDistributed(nn.Linear(hidden_size*2, input_size))
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size//2),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden_size//2, output_size)
        )

        self.actv = nn.ReLU()

    def forward(self, x):
        decoded_output1, _ = self.decoder1(x)   # (3, 10, 64)
        decoded_output2, _ = self.decoder2(decoded_output1) # (3, 10, 128)
        # distributer_output = self.timedistributed(decoded_output2)  # (3, 10, 2)
        # Only use the last output sequence
        last_output = decoded_output2[:, -1, :]

        # Fully connected layer
        output = self.actv(self.fc(last_output))

        return output.squeeze(1)


class LSTM_Autoencoder(nn.Module):
    def __init__(self, input_size=5, hidden_size=512, sequence_length=5, batch_size=CFG['BATCH_SIZE'], output_size=CFG['PREDICT_SIZE']):
        super(LSTM_Autoencoder, self).__init__()
        self.encoder = Encoder(input_size, hidden_size, sequence_length, batch_size)
        self.decoder = Decoder(input_size, hidden_size, output_size)

    def forward(self, x):
        encoded_output = self.encoder(x)
        decoded_output = self.decoder(encoded_output)

        return decoded_output

class BaseModel(nn.Module):
    def __init__(self, input_size=5, hidden_size=512, output_size=CFG['PREDICT_SIZE']):
        super(BaseModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)

        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size//2),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(hidden_size//2, output_size)
        )

        self.actv = nn.ReLU()

    def forward(self, x):
        # x shape: (B, TRAIN_WINDOW_SIZE, 5)
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size, x.device)

        # LSTM layer
        lstm_out, _ = self.lstm(x, hidden)
        lstm_out2, hidden = self.lstm2(lstm_out)

        # Only use the last output sequence
        last_output = lstm_out[:, -1, :]

        # Fully connected layer
        output = self.actv(self.fc(last_output))

        return output.squeeze(1)

    def init_hidden(self, batch_size, device):
        # Initialize hidden state and cell state
        return (torch.zeros(1, batch_size, self.hidden_size, device=device),
                torch.zeros(1, batch_size, self.hidden_size, device=device))

모델 학습

In [33]:
def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = 9999999
    best_model = None

    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        train_mae = []
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)

            optimizer.zero_grad()

            output = model(X)
            loss = criterion(output, Y)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        val_loss = validation(model, val_loader, criterion, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')

        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
            print('Model Saved')
    return best_model

In [34]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []

    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.to(device)
            Y = Y.to(device)

            output = model(X)
            loss = criterion(output, Y)

            val_loss.append(loss.item())
    return np.mean(val_loss)

Run !!

In [35]:
model = LSTM_Autoencoder()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
infer_model = train(model, optimizer, train_loader, val_loader, device)

  0%|          | 0/1412 [00:00<?, ?it/s]

  0%|          | 0/353 [00:00<?, ?it/s]

Epoch : [1] Train Loss : [0.02997] Val Loss : [0.02962]
Model Saved


  0%|          | 0/1412 [00:00<?, ?it/s]

  0%|          | 0/353 [00:00<?, ?it/s]

...

  0%|          | 0/353 [00:00<?, ?it/s]

Epoch : [4] Train Loss : [0.02835] Val Loss : [0.02962]
Model Saved


  0%|          | 0/1412 [00:00<?, ?it/s]

  0%|          | 0/353 [00:00<?, ?it/s]

Epoch : [5] Train Loss : [0.02714] Val Loss : [0.02979]


모델 추론

In [36]:
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0, drop_last=True)

In [37]:
def inference(model, test_loader, device):
    predictions = []

    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.to(device)

            output = model(X)

            # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
            output = output.cpu().numpy()

            predictions.extend(output)

    return np.array(predictions)

In [38]:
pred = inference(infer_model, test_loader, device)

  0%|          | 0/5 [00:00<?, ?it/s]

In [39]:
# 추론 결과를 inverse scaling
for idx in range(len(pred)):
    pred[idx, :] = pred[idx, :] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]

# 결과 후처리
pred = np.round(pred, 0).astype(int)

In [40]:
pred.shape

(15890, 21)

Submission

In [41]:
submit = pd.read_csv('./drive/MyDrive/sample_submission.csv')
submit.head()

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
submit.iloc[:,1:] = pred
submit.head()

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,1,1,1,1,1
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
submit.to_csv('./drive/MyDrive/baseline_submit.csv', index=False)